In [1]:
import pandas as pd
import sqlalchemy
from matplotlib import pyplot as plt
import datetime
import copy
import numpy as np


In [2]:
db = sqlalchemy.engine.url.URL(drivername='mysql',
                               host='127.0.0.1',
                               database='sherman_1',
                               query={'read_default_file': '~/.my.cnf', 'charset': 'utf8'})
engine = sqlalchemy.create_engine(db)

In [3]:
df = pd.read_sql('''
    SELECT * FROM sherman_1.locations;
''' ,engine)

### Feature Analysis
* double_latitude -> the location’s latitude, in degrees
* double_longitude -> the location’s longitude, in degrees
    * will be merged to get location
* double_bearing -> the location’s bearing, in degrees
* double_speed -> the speed if available, in meters/second over ground
    * will be removed, not helpful in this analysis
* double_altitude -> if available, in meters above sea level
    * will be removed, not available for all the records

In [5]:
df['date'] = pd.to_datetime(df['timestamp'],unit='ms').dt.date

In [6]:
df['location'] = df['double_latitude'].astype(str) + "," + df['double_longitude'].astype(str)

In [7]:
df_less = df[['device_id', 'location', 'date']]

In [8]:
df_less

,device_id,location,date
0,39f277f4-a04c-4668-8a36-8addb0a0265f,"42.358840942383,-71.050247192383",2022-11-07
1,39f277f4-a04c-4668-8a36-8addb0a0265f,"42.357767601369,-71.050605050462",2022-11-07
2,39f277f4-a04c-4668-8a36-8addb0a0265f,"42.358840942383,-71.050247192383",2022-11-07
3,39f277f4-a04c-4668-8a36-8addb0a0265f,"42.35779482303,-71.050588828588",2022-11-07
4,39f277f4-a04c-4668-8a36-8addb0a0265f,"42.358840942383,-71.050247192383",2022-11-07
...,...,...,...
2527653,30ca0662-67d1-42d6-9610-d435e60a6a79,"39.022381296269,-84.727613200974",2023-01-11
2527654,30ca0662-67d1-42d6-9610-d435e60a6a79,"39.022381296269,-84.727613200974",2023-01-11
2527655,30ca0662-67d1-42d6-9610-d435e60a6a79,"39.022381296269,-84.727613200974",2023-01-11
2527656,30ca0662-67d1-42d6-9610-d435e60a6a79,"39.022381296269,-84.727613200974",2023-01-11


### Group by
Group by 'device_id' and 'date' aggregate number of unique and tola of palces visited

In [9]:
df_output = df_less.groupby(['device_id','date']).agg({'location':['nunique','count']})

In [11]:
df_output = df_less.groupby(['date']).agg({'device_id':['nunique','count']})

In [12]:
df_output

device_id       
             nunique  count
date                       
2021-04-03         1    388
2021-06-23         1     70
2021-12-15         1      2
2022-02-09         1    506
2022-02-10         1    956
...              ...    ...
2023-01-07        67  58025
2023-01-08        67  58691
2023-01-09        66  58282
2023-01-10        62  53701
2023-01-11        60  45804

[340 rows x 2 columns]

In [13]:
df_output = df_output.reset_index()

In [14]:
df_output.columns = ['device_id', 'date', 'nunique','count']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [ ]:
df_output.to_csv("locations.csv", index = False)

### Filter 28 days

In [ ]:
def find_first_day(device_id, df_cleaned):
    df_target = df_cleaned[df_cleaned['device_id']==device_id]
    return min(df_target['date'])

In [ ]:
def get_1_to_28(df_cleaned):
    first_day = {device_id:find_first_day(device_id, df_cleaned) 
                 for device_id in set(df_cleaned['device_id'].tolist())
                if find_first_day(device_id, df_cleaned)>=datetime.date(year=2022,month=11,day=11)}
    df_cleaned = df_cleaned[df_cleaned['device_id'].isin(first_day)]
    df_cleaned['day'] = df_cleaned.apply(lambda x: 1+(x['date']-first_day[x['device_id']]).days, axis = 1)
    df_cleaned = df_cleaned[df_cleaned['day']<=28]
    return df_cleaned

In [ ]:
first_day = {device_id:find_first_day(device_id, df_output) for device_id in set(df_output['device_id'].tolist())}

In [ ]:
df_cleaned_1_28 = get_1_to_28(df_output)
df_cleaned_1_28

In [ ]:
df_plot = copy.deepcopy(df_cleaned_1_28)
df_plot = df_plot[['nunique','count','day']]
df_plot = df_plot.groupby('day').mean()
df_plot = df_plot.reset_index()

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=300)
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
x = [i+1 for i in range(28)]
y1 = df_plot['nunique'].tolist()
y2 = df_plot['count'].tolist()
ln1 = ax1.plot(x,y1,label = 'nunique', color = 'red')
ln2 = ax1.plot(x,y2,label = 'count', color = 'blue')
plt.xlim(1,28)
plt.xticks([i+1 for i in range(28)],[i+1 for i in range(28)])
plt.title("Locations")
plt.xlabel("Days After Join")
ax1.set_ylabel('nunique', color = 'red')
ax2.set_ylabel('count', color = 'blue')
lns = ln1+ln2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0)
plt.show()

In [ ]:
df_id = pd.read_csv("/data/meditation/majid/tables/id.csv")
df_id = df_id.drop_duplicates('study_id')
df_id = df_id.drop('id', axis = 1)
df_id.columns = ['device_id','Study ID']

df_group = pd.read_csv('/data/meditation/majid/surveys/0.csv', skiprows=[1,2])[['Study ID','Group']]
df_group = df_id.merge(df_group, on = 'Study ID', how = 'left')
df_group

In [ ]:
df_plot = copy.deepcopy(df_cleaned_1_28)
df_plot = df_plot.merge(df_group[['device_id','Group']], on = 'device_id', how = 'left')
df_plot = df_plot[['nunique','day','Group']]
df_plot = df_plot.groupby(['Group','day']).mean()
df_plot = df_plot.reset_index()
df_plot

In [ ]:
plt.figure(figsize=(8, 6), dpi=300)
for group in ['Active Control', 'Experimental Condition', 'Passive Control']:
    df_target = df_plot[df_plot['Group']==group]
    x = [i+1 for i in range(28)]
    y = df_target["nunique"].tolist()
    plt.plot(x,y,label = group)
plt.xlim(1,28)
plt.xticks([i+1 for i in range(28)],[i+1 for i in range(28)])
plt.title("Locations")
plt.xlabel("Days After Join")
plt.ylabel('Unique Places Visited')
plt.legend()
plt.show()